In [12]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(r'D:\pythonWorkspace\TestForFood\my_model')
tflite_model = converter.convert()
open("saved_model/my_model.tflite", "wb").write(tflite_model)

converter = tf.lite.TFLiteConverter.from_saved_model(r'D:\pythonWorkspace\TestForFood\my_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
open("saved_model/my_model.quant.tflite", "wb").write(tflite_quant_model)

44216688

In [15]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf
import keras

model = tf.keras.models.load_model(r'D:\pythonWorkspace\TestForFood\Food-Image-Recognition/best_model_101class.hdf5')
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
quantize_layer_1 (QuantizeLayer (None, None, None, 3 3           input_1[0][0]                    
__________________________________________________________________________________________________
quant_conv2d (QuantizeWrapper)  (None, None, None, 3 929         quantize_layer_1[1][0]           
__________________________________________________________________________________________________
quant_batch_normalization (Quan (None, None, None, 3 97          quant_conv2d[0][0]               
_______________________________________________________________________________________

In [28]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

KeyboardInterrupt: 

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
tar: Error opening archive: Failed to open 'food-101.tar.gz'


In [22]:
from keras.preprocessing.image import ImageDataGenerator

img_rows,img_cols = 200, 200

train_data_dir = r'D:\pythonWorkspace\COVID_DATA\collection-re2\stacking'
validation_data_dir = r'D:\pythonWorkspace\COVID_DATA\collection-re2\fuzzyColor'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
nb_train_samples = 3036 
nb_validation_samples = 3036 


train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

Found 3036 images belonging to 3 classes.
Found 3036 images belonging to 3 classes.


In [26]:
q_aware_model.fit(train_generator,
            batch_size=32,
            epochs=2,
            validation_data=validation_generator,
            )

Epoch 1/2


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,101] and labels shape [96]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-23-a16cb30f0851>:5) ]] [Op:__inference_train_function_621091]

Function call stack:
train_function


In [27]:
_, baseline_model_accuracy = model.evaluate(
    train_generator, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   train_generator, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,101] labels_size=[32,3]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-27-4238288b4369>:2) ]] [Op:__inference_test_function_623030]

Function call stack:
test_function
